In [1]:
%pip install -qU python-dotenv langchain langchain-openai
%pip install --upgrade langchain langchain-core
%pip install --upgrade pydantic

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
### Calling OpenAI API Directly

import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env if present
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY not set. Set it in your environment or in a .env file.")

client = OpenAI()

llm_model = "gpt-4o-mini"

# Sample call to verify the client works
resp = client.responses.create(
    model=llm_model,
    input="Tell me a short story about a frog.",
    temperature=0.7,
    max_output_tokens=100    
)
print(resp.output_text)

Once upon a time in a lush, green pond, there lived a curious little frog named Freddie. Unlike the other frogs, who were content to hop from lily pad to lily pad, Freddie dreamed of exploring the world beyond the water’s edge.

One sunny morning, he decided it was time for an adventure. With a leap and a splash, he ventured onto the soft, mossy bank. As he explored, he met a wise old turtle named Tilly.

“Where are you off to,


In [3]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# Define the prompt template
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```"""

# Create a ChatPromptTemplate with a single HumanMessagePromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(template_string)
])

# Define input variables
customer_style = """American English \
in a calm and respectful tone"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# Format the prompt into messages
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

response = chat.invoke(customer_messages)
response


AIMessage(content='I’m quite frustrated that the lid of my blender came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn’t cover the cost of cleaning up my kitchen. I would really appreciate your assistance with this issue. Thank you!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 86, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZM9EHJPG0zp2i2KRVQgpUml5w0Db', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--1f20a48f-7dc4-4e30-b99d-2bff49e97e7f-0', usage_metadata={'input_tokens': 86, 'output_tokens': 53, 'total_tokens': 139, 'input_token_

In [5]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

service_response = chat.invoke(service_messages)
print(service_response)

content="Ahoy, me hearty! I be sorry to inform ye that the warranty be not coverin' the expenses fer cleanin' yer galley, as it seems ye may have misused yer blender by forgettin' to secure the lid afore settin' it to churn. Aye, 'tis a bit of tough luck, indeed! Fair winds to ye, and may yer future voyages be smoother!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 78, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZM9G2eH2tfKAKsgoBJE8qf5izwh5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--b48932b6-98f6-40cf-936a-30d958625239-0' usage_metadata={'input_tokens': 78, 'output_tokens': 82,

In [6]:
### Output Parsing

{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
)

prompt_template = ChatPromptTemplate.from_template(review_template)

review = prompt_template.format_messages(text=customer_review)
review_response = chat.invoke(review)
print(review_response.content)


{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there",
    "I think it's worth it for the extra features"
  ]
}


In [12]:
### Parse the LLM output string into a Python dictionary

from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from typing import List

# Define the schema using Pydantic
class ReviewSchema(BaseModel):
    gift: bool = Field(..., description="Was the item purchased as a gift for someone else?")
    delivery_days: int = Field(..., description="How many days did it take for the product to arrive? If this information is not found, output -1.")
    price_value: List[str] = Field(..., description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

# Create the parser
output_parser = PydanticOutputParser(pydantic_object=ReviewSchema)

# Print format instructions
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"gift": {"description": "Was the item purchased as a gift for someone else?", "title": "Gift", "type": "boolean"}, "delivery_days": {"description": "How many days did it take for the product to arrive? If this information is not found, output -1.", "title": "Delivery Days", "type": "integer"}, "price_value": {"description": "Extract any sentences about the value or price, and output them as a comma separated Python list.", "items": {"type": "string"}, "title": "Price Value", "type": "array"}}, "required": ["gift", "delivery_day

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)
                            
response = chat.invoke(messages)
print(response.content) 
print(type(response.content)) #ReviewSchema object


{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [29]:
output_parser.parse_with_prompt(response.content, messages)
output_dict.gift

True